In [1]:
# EDA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

sns.set_style("whitegrid")

# Machine Learning
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

### Carregar os dados

In [2]:
# Carregar os dados
df_leads = pd.read_csv('../datasets/leads_cleaned.csv')

In [3]:
# Mostrar as primeiras linhas
df_leads.head()

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,A free copy of Mastering The Interview,Last Notable Activity
0,API,Olark Chat,0,0,0,0.0,0,0.0,Page Visited on Website,0,0,0,0,0,0,0,Modified
1,API,Organic Search,0,0,0,5.0,674,2.5,Email Opened,0,0,0,0,0,0,0,Email Opened
2,Landing Page Submission,Direct Traffic,0,0,1,2.0,1532,2.0,Email Opened,0,0,0,0,0,0,1,Email Opened
3,Landing Page Submission,Direct Traffic,0,0,0,1.0,305,1.0,Unreachable,0,0,0,0,0,0,0,Modified
4,Landing Page Submission,Google,0,0,1,2.0,1428,1.0,Converted to Lead,0,0,0,0,0,0,0,Modified


In [4]:
# Mostrar os últimas linhas
df_leads.tail()

,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Search,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,A free copy of Mastering The Interview,Last Notable Activity
9069,Landing Page Submission,Direct Traffic,1,0,1,8.0,1845,2.67,Email Marked Spam,0,0,0,0,0,0,0,Email Marked Spam
9070,Landing Page Submission,Direct Traffic,0,0,0,2.0,238,2.00,SMS Sent,0,0,0,0,0,0,1,SMS Sent
9071,Landing Page Submission,Direct Traffic,1,0,0,2.0,199,2.00,SMS Sent,0,0,0,0,0,0,1,SMS Sent
9072,Landing Page Submission,Google,0,0,1,3.0,499,3.00,SMS Sent,0,0,0,0,0,0,0,SMS Sent
9073,Landing Page Submission,Direct Traffic,0,0,1,6.0,1279,3.00,SMS Sent,0,0,0,0,0,0,1,Modified


In [5]:
# Mostrar a estrutura do dataset
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9074 entries, 0 to 9073
Data columns (total 17 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Lead Origin                             9074 non-null   object 
 1   Lead Source                             9074 non-null   object 
 2   Do Not Email                            9074 non-null   int64  
 3   Do Not Call                             9074 non-null   int64  
 4   Converted                               9074 non-null   int64  
 5   TotalVisits                             9074 non-null   float64
 6   Total Time Spent on Website             9074 non-null   int64  
 7   Page Views Per Visit                    9074 non-null   float64
 8   Last Activity                           9074 non-null   object 
 9   Search                                  9074 non-null   int64  
 10  Newspaper Article                       9074 non-null   int6

### Preparação dos dados

In [6]:
# Preparar os dados para o modelo
X = df_leads.drop(columns=['Converted'])
y = df_leads['Converted']

In [7]:
# Criar lista de colunas para a parte de interpretabilidade
numeric_features = X.select_dtypes(include=['number']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [8]:
# Usar o preprocessor já salvo anteriormente
import joblib
preprocessor = joblib.load('../preprocessor_dataset_leads.pkl')

In [9]:
# Dividir os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=51)

In [10]:
# Aplicar o preprocessor
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [11]:
# Mostrar a estrutura destes conjuntos
print(X_train.shape)
print(X_test.shape)

(7259, 68)
(1815, 68)


### Treinamento do Modelo

In [12]:
# Criar o modelo de BoostingClassifier
boosting_model = AdaBoostClassifier(
  estimator=LogisticRegression(),
  n_estimators=50,
  learning_rate=1,
  random_state=51
)

In [13]:
# Treinar o modelo
boosting_model.fit(X_train, y_train)

/home/emerson/.local/share/virtualenvs/edtech-yAn4fxuW/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=LogisticRegression(), learning_rate=1,
                   random_state=51)

### Avaliação do Modelo

In [14]:
# Fazer predições no conjunto de testes
y_pred = boosting_model.predict(X_test)

In [15]:
# Métricas do modelo
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [16]:
# Mostrar métricas
print(f'Acurácia: {accuracy}')
print(f'Precisão: {precision}')
print(f'Recall: {recall}')
print(f'F1: {f1}')

Acurácia: 0.7928374655647383
Precisão: 0.7441860465116279
Recall: 0.6686567164179105
F1: 0.7044025157232704


In [17]:
# Mostrar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_pred)

fig = px.imshow(
  conf_matrix,
  labels=dict(
    x='Predição',
    y='Real',
    color='Contagem'
  ),
  x=['Not Converted', 'Converted'],
  y=['Not Converted', 'Converted'],
  color_continuous_scale='Viridis'
)

fig.update_traces(
  text=conf_matrix,
  texttemplate='%{z}'
)
fig.update_layout(coloraxis_showscale=False)

fig.show()

In [18]:
# Calcular a importância das variáveis
importances = np.mean([np.abs(estimator.coef_[0]) for estimator in boosting_model.estimators_], axis=0)

In [19]:
# Obter os nomes reais das features reais
feature_names = (
  numeric_features.tolist() +
  preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
)

In [20]:
# Criar um Dataframe combinando os nomes das variáveis com as importâncias (coeficientes)
df_feature_importances = pd.DataFrame(
  {
    'Feature': feature_names,
    'Importance': importances
  }
)

In [21]:
# Ordenar Dataframe pela importância
df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=True)

In [22]:
# Plotar o gráfico de importância
fig = px.bar(
  df_feature_importances,
  x='Importance',
  y='Feature',
  orientation='h',
  title='Importância das Features (baseada nos coeficientes absolutos)'
)

fig.update_layout(
  height=1280,
  width=1000,
  yaxis={'categoryorder': 'total ascending'}
)

fig.show()

### Saídas do Modelo

In [23]:
# Erros dos estimadores
boosting_model.estimator_errors_

array([0.37994214, 0.2901924 , 0.2633916 , 0.2802357 , 0.29150886,
       0.30945804, 0.32234253, 0.33561959, 0.34674777, 0.35809818,
       0.36836696, 0.37674798, 0.38378613, 0.39237038, 0.39880113,
       0.40672294, 0.41218244, 0.41639987, 0.41965685, 0.42466887,
       0.42708748, 0.42511308, 0.42723245, 0.42872637, 0.43171125,
       0.43206317, 0.43303061, 0.43175052, 0.43451599, 0.4288492 ,
       0.43200229, 0.4332551 , 0.43373651, 0.43256044, 0.43538201,
       0.43367988, 0.43574517, 0.43545941, 0.43545561, 0.43485606,
       0.43683408, 0.43795997, 0.43844451, 0.43690541, 0.43582208,
       0.43497456, 0.43625688, 0.4359964 , 0.4414345 , 0.43807825])

In [24]:
# Pesos dos estimadores
boosting_model.estimator_weights_

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [25]:
# Predizer a Probabilidade de Conversão
y_pred_prob = boosting_model.predict_proba(X_test)

In [26]:
# Visualizar Probabilidades
y_pred_prob

array([[0.49286509, 0.50713491],
       [0.50550288, 0.49449712],
       [0.50550288, 0.49449712],
       ...,
       [0.50957745, 0.49042255],
       [0.52071227, 0.47928773],
       [0.48323981, 0.51676019]])